## 初期設定

In [1]:
"""環境"""
# .lab

"""更新履歴"""
# 最終更新日：2025/08/12
# 2025/08/12 inputsは使わないことにしました。
# 2025/08/04 cachedirの洗浄をオプションにしました。
# 2025/07/22 pickleを保存するために、pickleとdatetimeをインポートするようにしました。
# 2025/07/17 inputsを保存するjsonを作ることにしました
# 2025/07/17 ctypesを使ってtkinterの解像度を上げることにしました
# 2025/05/21 shutilを使って、cachedirを洗浄することにしました
# 2025/05/15 simple_progress_barおよびdata(dict)を使い始めました。
# 2025/02/25 os.getcwd()を採用し、__file__を定義する必要をなくしました。

"""モジュール読み込み"""
# ファイル操作等
import sys
import os
from datetime import datetime
from pprint import pprint
import logging
import pickle
# import struct
from tqdm import tqdm
import h5py
# import threading
import json
import shutil

# tkinter
from tkinter import filedialog, messagebox, Tk
import ctypes
ctypes.windll.shcore.SetProcessDpiAwareness(1) # tkinterの解像度を上げる

# データ分析ツール
import pandas as pd
import numpy as np
import scipy as sp
# import math
# from sklearn.linear_model import LinearRegression

# グラフ等作成用
import matplotlib
import matplotlib.pyplot as plt         # 図の作成用
from PIL import Image as im
# import cv2
from IPython.display import display, HTML, clear_output, update_display, Image

# 自作モジュール
sys.path.append(r"C:\Users\okaza\pythonenv")
from modules.Mytools.Tools import print_fileinfo, h5_tree, dict_tree, simple_progress_bar, clean_cache_except_logfiles, get_total_size
import modules.Mytools.Settings
import modules.fitXRD as fx
from modules.peakfit import peakfit, pseudoVoigt

"""ログ管理ツール作成"""
# chche directoryの設定
cachedir = os.path.abspath(os.getcwd() + "/.cache")
if False:
    clean_cache_except_logfiles(cachedir)
os.makedirs(cachedir, exist_ok=True)

# loggerの作成
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
format = "%(levelname)-9s  %(asctime)s [%(filename)s:%(lineno)d] %(message)s"

# Streamハンドラクラスを作成
sh = logging.StreamHandler()
sh.setLevel(logging.DEBUG)
sh.setFormatter(logging.Formatter(format))
logger.addHandler(sh)

# Fileハンドラクラスをインスタンス化
logfile = cachedir + ""
fh = logging.FileHandler(filename=cachedir + "/notebook.log", encoding="utf-8")
fh.setLevel(logging.DEBUG)
fh.setFormatter(logging.Formatter(format))
logger.addHandler(fh)
logger.debug("[Activate workspace]: " + os.getcwd())
logger.debug("[Set cache]: "+ os.path.abspath(cachedir))
logger.debug("[Activate log]: " + os.path.abspath(cachedir + "/notebook.log"))

# globalなデータを作成
data = dict()
logger.debug("[Create global variable]: data")

DEBUG      2025-08-12 11:55:00,803 [2155881028.py:80] [Activate workspace]: c:\Users\okaza\pythonenv\fpd\fileinfo
DEBUG      2025-08-12 11:55:00,804 [2155881028.py:81] [Set cache]: c:\Users\okaza\pythonenv\fpd\fileinfo\.cache
DEBUG      2025-08-12 11:55:00,805 [2155881028.py:82] [Activate log]: c:\Users\okaza\pythonenv\fpd\fileinfo\.cache\notebook.log
DEBUG      2025-08-12 11:55:00,805 [2155881028.py:86] [Create global variable]: data


生データの保存

In [ ]:
def save_pickle():

    """変更履歴
    * 2025/08/05: defaultextensitionを追加、initialdirの削除
    * 2025/08/01: 作成ののち変更
    """

    # 現在時刻の取得
    dt = datetime.now()
    initfilename = dt.strftime('%Y%m%d%H%M%S%f') + ".pkl"

    # tkinterでファイルダイアログを開く
    window = Tk()
    window.wm_attributes("-topmost", 1)
    window.withdraw()
    filename = filedialog.asksaveasfilename(
        parent = window,
        filetypes = [
            ("pkl", "*.pkl"),
        ],
        initialfile=initfilename,
        defaultextension=".pkl",
    )

    # filenameがセットされたら保存する
    if not (filename == ""):
        with open(filename, mode = "wb") as f:
            pickle.dump(
                obj = data,
                file = f
            )
    logger.info("[Save data]: " + os.path.abspath(filename))

    return
save_pickle()
del save_pickle

生データの読み込み

In [ ]:
def load_pickle():

    """変更履歴
    * 2025/08/04: dataの上書きができていませんでした
    * 2025/07/25: 作成
    """

    # tkinterでファイルダイアログを開く
    window = Tk()
    window.wm_attributes("-topmost", 1)
    window.withdraw()
    filename = filedialog.askopenfilename(
        parent = window,
        filetypes = [
            ("pkl", "*.pkl"),
        ],
    )
    global data
    if not (filename == ""):
        with open(filename, mode = "rb") as f:
            data = pickle.load(
                file = f
            )
    logger.info("[Load]: " + os.path.abspath(filename))
    dict_tree(data)

    return
load_pickle()
del load_pickle

## 目的

ファイル情報で時系列データを扱えるのかどうかを確認します。

In [8]:
def set_filelist():

    # ディレクトリ名を指定
    dir = r"C:\Users\okaza\Box\DataStrorage\SPring8-2025-06\Okazaki\UODE15\FPD_1"
    
    # ヘッダーとフッターを指定
    header = "UODE15_5_"
    footer = ".his"

    if True: # Main

        """変更履歴
        * 2025/08/12: inputsを削除
        * 2025/08/01: 変更履歴作成
        """

        # ヘッダーとフッターを含むファイル名を取得
        flist = list()
        for __ in os.listdir(dir):
            if not header in __:
                continue
            if not footer in __:
                continue
            flist.append(__)
        logger.debug("Read filenames")

        # ソート
        flist.sort(key = (lambda x: int(x.replace(header, "").replace(footer, ""))))
        logger.debug("Sort the file name list")

        # 表示
        flist = flist[:442]
        for f in flist:
            print(os.path.abspath(dir + "/" + f))

        # 格納
        key = sys._getframe().f_code.co_name
        data[key] = dict()
        data[key]["dir"] = dir
        data[key]["flist"] = flist
        data[key]["header"] = header
        data[key]["footer"] = footer
        logger.info("[Add variables]: " + key)
        dict_tree(data)

    return
set_filelist()
del set_filelist

DEBUG      2025-08-12 12:01:07,909 [4156246839.py:25] Read filenames
DEBUG      2025-08-12 12:01:07,911 [4156246839.py:29] Sort the file name list
INFO       2025-08-12 12:01:07,913 [4156246839.py:43] [Add variables]: set_filelist


C:\Users\okaza\Box\DataStrorage\SPring8-2025-06\Okazaki\UODE15\FPD_1\UODE15_5_0.his
C:\Users\okaza\Box\DataStrorage\SPring8-2025-06\Okazaki\UODE15\FPD_1\UODE15_5_1.his
C:\Users\okaza\Box\DataStrorage\SPring8-2025-06\Okazaki\UODE15\FPD_1\UODE15_5_2.his
C:\Users\okaza\Box\DataStrorage\SPring8-2025-06\Okazaki\UODE15\FPD_1\UODE15_5_3.his
C:\Users\okaza\Box\DataStrorage\SPring8-2025-06\Okazaki\UODE15\FPD_1\UODE15_5_4.his
C:\Users\okaza\Box\DataStrorage\SPring8-2025-06\Okazaki\UODE15\FPD_1\UODE15_5_5.his
C:\Users\okaza\Box\DataStrorage\SPring8-2025-06\Okazaki\UODE15\FPD_1\UODE15_5_6.his
C:\Users\okaza\Box\DataStrorage\SPring8-2025-06\Okazaki\UODE15\FPD_1\UODE15_5_7.his
C:\Users\okaza\Box\DataStrorage\SPring8-2025-06\Okazaki\UODE15\FPD_1\UODE15_5_8.his
C:\Users\okaza\Box\DataStrorage\SPring8-2025-06\Okazaki\UODE15\FPD_1\UODE15_5_9.his
C:\Users\okaza\Box\DataStrorage\SPring8-2025-06\Okazaki\UODE15\FPD_1\UODE15_5_10.his
C:\Users\okaza\Box\DataStrorage\SPring8-2025-06\Okazaki\UODE15\FPD_1\UODE15

ctimeを取得します。

In [15]:
def get_fpd_ctime():

    d = data["set_filelist"]["dir"]
    for f in data["set_filelist"]["flist"]:
        filename = os.path.join(d, f)

        creation_time = datetime.fromtimestamp(os.path.getmtime(filename))


        format = '%Y/%m/%d %H:%M:%S.%f'
        print(datetime.strftime(creation_time, format))

    return
get_fpd_ctime()
del get_fpd_ctime

2025/06/18 05:54:54.000000
2025/06/18 05:54:56.000000
2025/06/18 05:54:56.000000
2025/06/18 05:54:58.000000
2025/06/18 05:54:58.000000
2025/06/18 05:55:00.000000
2025/06/18 05:55:00.000000
2025/06/18 05:55:02.000000
2025/06/18 05:55:02.000000
2025/06/18 05:55:04.000000
2025/06/18 05:55:04.000000
2025/06/18 05:55:06.000000
2025/06/18 05:55:06.000000
2025/06/18 05:55:08.000000
2025/06/18 05:55:08.000000
2025/06/18 05:55:10.000000
2025/06/18 05:55:10.000000
2025/06/18 05:55:12.000000
2025/06/18 05:55:12.000000
2025/06/18 05:55:14.000000
2025/06/18 05:55:14.000000
2025/06/18 05:55:16.000000
2025/06/18 05:55:16.000000
2025/06/18 05:55:18.000000
2025/06/18 05:55:18.000000
2025/06/18 05:55:20.000000
2025/06/18 05:55:20.000000
2025/06/18 05:55:22.000000
2025/06/18 05:55:22.000000
2025/06/18 05:55:24.000000
2025/06/18 05:55:24.000000
2025/06/18 05:55:26.000000
2025/06/18 05:55:26.000000
2025/06/18 05:55:28.000000
2025/06/18 05:55:28.000000
2025/06/18 05:55:30.000000
2025/06/18 05:55:30.000000
2

最小単位は秒か。。。  
時系列データを扱うには精度が悪いね。